<a href="https://colab.research.google.com/github/akktrsst/Llamaindex/blob/master/Llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#### Text-to-SQL Guide

In [1]:
%pip install llama-index-llms-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00


In [2]:
# !pip install llama-index

In [3]:
import os
import openai

In [4]:
#OPENAI_API_KEY='sk-proj-IWmFpaWTLdfrlDbHq9zKT3BlbkFJrH2YsxJcZDwUt70CEUx4'

In [5]:
os.environ["OPENAI_API_KEY"] = "sk-proj-IWmFpaWTLdfrlDbHq9zKT3BlbkFJrH2YsxJcZDwUt70CEUx4"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [6]:
from IPython.display import Markdown, display

In [7]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)

In [8]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [9]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

In [10]:
city_stats_table

Table('city_stats', MetaData(), Column('city_name', String(length=16), table=<city_stats>, primary_key=True, nullable=False), Column('population', Integer(), table=<city_stats>), Column('country', String(length=16), table=<city_stats>, nullable=False), schema=None)

In [11]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI

In [12]:
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")

In [13]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [14]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {
        "city_name": "Chicago",
        "population": 2679000,
        "country": "United States",
    },
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [15]:
city_stats_table

Table('city_stats', MetaData(), Column('city_name', String(length=16), table=<city_stats>, primary_key=True, nullable=False), Column('population', Integer(), table=<city_stats>), Column('country', String(length=16), table=<city_stats>, nullable=False), schema=None)

In [16]:
cursor

In [17]:
# view current table
stmt = select(
    city_stats_table.c.city_name,
    city_stats_table.c.population,
    city_stats_table.c.country,
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea')]


In [18]:
### Query Index

In [19]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text("SELECT city_name from city_stats"))
    for row in rows:
        print(row)

('Chicago',)
('Seoul',)
('Tokyo',)
('Toronto',)


### Part 1: Text-to-SQL Query Engine

In [21]:
pip install llama-index-embeddings-openai

In [22]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["city_stats"], llm=llm
)
query_str = "Which city has the highest population?"
response = query_engine.query(query_str)

In [23]:
display(Markdown(f"<b>{response}</b>"))

<b>Tokyo has the highest population among all cities, with a population of 13,960,000.</b>

### Part 2: Query-Time Retrieval of Tables for Text-to-SQL

In [24]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

In [25]:
response = query_engine.query("Which city has the highest population?")
display(Markdown(f"<b>{response}</b>"))

<b>Tokyo has the highest population among all cities, with a population of 13,960,000.</b>

### Text to Pandas

In [27]:
!pip install llama-index llama-index-experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 36.8 MB/s eta 0:00:00


In [30]:
import logging
import sys
from IPython.display import Markdown, display

import pandas as pd
from llama_index.experimental.query_engine import PandasQueryEngine


logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [31]:
# Test on some sample data
df = pd.DataFrame(
    {
        "city": ["Toronto", "Tokyo", "Berlin"],
        "population": [2930000, 13960000, 3645000],
    }
)

In [32]:
query_engine = PandasQueryEngine(df=df, verbose=True)

In [33]:
response = query_engine.query(
    "What is the city with the highest population?",
)

> Pandas Instructions:
```
df.loc[df['population'].idxmax()]['city']
```
> Pandas Output: Tokyo


In [34]:
display(Markdown(f"{response}"))

Tokyo

In [36]:
# get pandas python instructions
print(response.metadata["pandas_instruction_str"])

df.loc[df['population'].idxmax()]['city']


In [37]:
query_engine = PandasQueryEngine(df=df, verbose=True, synthesize_response=True)
response = query_engine.query(
    "What is the city with the highest population? Give both the city and population",
)
print(str(response))

> Pandas Instructions:
```
df.loc[df['population'].idxmax()]
```
> Pandas Output: city             Tokyo
population    13960000
Name: 1, dtype: object
The city with the highest population is Tokyo, with a population of 13,960,000.
